In [ ]:
#@markdown #**Welcome to PaperStack!**

#@markdown ### **What is PaperStack?**
#@markdown PaperStack is a **multi-agent AI pipeline** for **automated document generation**.
#@markdown It demonstrates how specialized AI agents can work together to produce **structured, domain-specific writing**—whether for research papers, technical reports, or other formal documents.

#@markdown This notebook is an **experiment in automation**, not an attempt to generate human-quality academic work.
#@markdown Instead, it provides a **transparent and modular** way to observe and tweak how AI systems compose structured content step by step.
#@markdown Since all outputs are fully AI-generated, they are **not attributable to any person or entity**.

#@markdown ---

#@markdown #**How It Works**

#@markdown ### **1. User Input**

#@markdown Once the user inputs a topic, the AI pipeline progresses through a structured sequence of tasks to produce, refine, and format the document.

#@markdown ### **2. AI Pipeline Overview**
#@markdown PaperStack operates through a structured multi-agent process, ensuring systematic content generation.

#@markdown **Task-Specific System Prompts**
#@markdown - Each AI agent is assigned a specialized role, receiving structured prompts tailored to its function within the pipeline. These task-specific instructions ensure focused execution at each stage.

#@markdown **Strict Formatting Guidelines for Outputs (JSON)**
#@markdown - Al output content adheres to standardized **JSON formatting**, maintaining consistency and ensuring compatibility for further processing.

#@markdown **Dual-Layer JSON Validation**
#@markdown - Generated JSON responses undergo **automated validation** to check for structural and syntactical correctness. If JSON parsing fails, the result is passed to a **LLM JSON fixing agent** to correct errors. This loop repeats until parsing is successul and the result is passed to the next agent in the pipeline.

#@markdown **Drafting and Paragraph-Level Revisions**
#@markdown - The system first drafts the document, then performs **paragraph-by-paragraph revisions**, enhancing clarity, coherence, and logical flow.

#@markdown **Section Scanning and Coherence Revisions**
#@markdown - After paragraph-level refinements, each section is analyzed for **coherence** and **repetition**. A separate AI agent provides targeted revision instructions for each to ensure logical consistency and reduce redundancy across the document.

#@markdown **Attribution Scanning and Citation Insertion**
#@markdown - The system scans the document to identify statements requiring attribution. Where necessary, citations are inserted in a structured format to maintain proper referencing.

#@markdown **Abstract Generation**
#@markdown - A summarization step creates a **concise abstract**, distilling the core arguments and conclusions of the document.

#@markdown **Formatting for Printing**
#@markdown - The final document is structured for readability and prepared for output in a format suitable for review.

In [ ]:
#@markdown ##**Future Plans**
#@markdown
#@markdown - Add more fields for user inputs, including:
#@markdown   - Additional context to guide the AI's understanding
#@markdown   - Additional fields of study
#@markdown   - Key points or arguments the user wants included
#@markdown   - User prompted hypotheses, methodology, data, and results
#@markdown   - Preferred philosophical frameworks or schools of thought
#@markdown   - Specific philosophers or works to reference
#@markdown   - Citation style preferences
#@markdown   - Option to include particular counterarguments or opposing views, and how to address them
#@markdown   - Writing style preference (academic, accessible, narrative)
#@markdown
#@markdown - Refine and improve the attribution scanner
#@markdown - Link to a database of works like Google Scholar and field-specific OAI-PMH databases to validate works cited
#@markdown - Incorporate specialized fine-tuned models for editing, revision, in-text attribution, and citation
#@markdown - Transfer to a web UI for improved UX

In [ ]:

#@markdown # **Getting Started**

#@markdown ###**Step 1: Get your API key.**

#@markdown PaperStack requires a **Together AI API Key** to generate content using the **Llama-3.3-70B-Instruct-Turbo-free** model.


#@markdown - Visit [**Together AI**](https://together.ai) and sign up or log in.
#@markdown - Navigate to **API Keys** in your account settings.
#@markdown - Click **Generate New Key**, then copy it.

#@markdown   ### **Step 2. Store Your API Key in Google Colab**
#@markdown - In the **left sidebar menu**, click the **key** icon .
#@markdown - Click **"Add a new secret"**.
#@markdown - In the **"Name"** field, enter:
#@markdown   `TogetherAPI`
#@markdown - In the **"Value"** field, paste your **API key**.
#@markdown - Toggle **"Notebook access"** to **ON**.
#@markdown - Press the  ▶ in the upper left corner of this cell.

#@markdown This notebook ensures **API key security** using Google Colab's **Secrets** feature.
#@markdown - API keys are **never displayed in outputs** or stored in the notebook.
#@markdown - If the notebook is **copied, shared, or downloaded**, **API keys do not transfer**.
#@markdown - Users must re-enter API credentials each session for security.
#@markdown
#@markdown For more information on how the Secrets feature works in Colab, refer to:
#@markdown [How to Use Secrets in Google Colab](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75)

from google.colab import userdata

KEY = userdata.get('TogetherAPI')
client = Together(api_key = KEY)


In [ ]:

#@markdown ## **Step 2: Initialize**

#@markdown Press the ▶ in the upper left corner of this cell to install Together and import libraries.

%%capture
!pip install together
from together import Together
import json
import re



In [ ]:
#@markdown ## **Step 3: Generate the Paper.**

Topic = "" #@param {"type":"string", "placeholder":"Type the topic or title"}

#@markdown Press the ▶ in the upper left corner of this to generate the paper. This could take 8-12 minutes, so please be patient.

def call_llm(prompt, system_prompt):
    """
    General function to call the LLM with a system prompt and user input.
    """
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=None,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>", "<|eom_id|>"],
        stream=False
    )
    return response.choices[0].message.content.strip()

def thesis_agent(topic):
    system_prompt = (
        "You are a philosophy professor specializing in academic philosophical writing. Your task is to generate a strong, clear, and well-reasoned thesis statement on the given topic. "
        "The thesis should be debatable, precise, and philosophically rigorous."
    )

    prompt = f"""
    Generate a thesis statement for the following topic:
    {topic}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "thesis": "string"
    }}

    Ensure the thesis is concise (one or two sentences) and presents a clear position that can be logically argued.
    """

    return call_llm(prompt, system_prompt)

def context_agent(thesis):
    system_prompt = (
        "You are a philosophy researcher specializing in historical and conceptual analysis in the field of philosophy. Your task is to identify key philosophical works, "
        "thinkers, and concepts relevant to the given thesis and summarize their relevance in relation to the argument. "
        "Your response must be formatted strictly as JSON and contain no extra text or explanations."
    )

    prompt = f"""
    Provide context for the following thesis:
    {thesis}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "philosophers": [
        {{
          "name": "string",
          "work": "string",
          "relevance": "string"
        }}
      ],
      "concepts": [
        {{
          "name": "string",
          "definition": "string",
          "relevance": "string"
        }}
      ]
    }}

    - Ensure at least two philosophers and two concepts are included.
    - Explanations must be concise yet specific, directly connecting each philosopher and concept to the thesis.
    """

    return call_llm(prompt, system_prompt)

def argument_agent(thesis):
    system_prompt = (
        "You are a formal logician. Your task is to construct a rigorous philosophical argument in support of the given thesis. "
        "Your response must follow formal logical principles, ensuring clear premises that lead to a reasoned conclusion. "
        "Additionally, you must present a counterargument and a refutation of that counterargument. "
        "Your response must be strictly formatted as JSON, without any extra text or explanation."
    )

    prompt = f"""
    Construct a structured argument based on the following thesis:
    {thesis}

    Your response must be formatted as a JSON object:

    {{
      "argument": {{
        "premises": [
          "string"
        ],
        "conclusion": "string"
      }},
      "counterargument": {{
        "premises": [
          "string"
        ],
        "conclusion": "string"
      }},
      "refutation": "string"
    }}

    Do not include any additional text before or after the JSON response.
    """

    return call_llm(prompt, system_prompt)

def discussion_agent(argument,context):
    system_prompt = (
        "You are a philosophy seminar leader facilitating an advanced discussion on the given argument in a graduate-level philosophy course. "
        "Your task is to generate a set of meaningful philosophical questions and accompanying exposition that critically explore, expand, and challenge the argument."
        "You and the other participants in the philosophy seminar have familiarized yourselves with the context."
    )

    prompt = f"""
    Discuss the following arguments in the appropriate context:

    Context:
    {context}

    Arguments:
    {argument}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "questions": [
        {{
          "id": "integer",
          "text": "string",
          "category": "string"
        }}
      ]
    }}

    Each question should be categorized under one of the following:
    - "validity" (questions about logical structure and consistency)
    - "soundness" (questions about the truth of premises)
    - "alternative perspectives" (questions that explore different viewpoints)
    - "implications" (questions about consequences of accepting the argument)

    Keep questions open-ended and specific enough to guide meaningful discussion, ensuring clarity and coherence.
    """

    return call_llm(prompt, system_prompt)

def overview_agent(thesis, arguments, context, discussion):
    system_prompt = (
      """You are an academic writer specializing in structuring philosophical essays for maximum clarity, coherence, and intellectual depth.

      Your task is to generate a comprehensive and well-organized overview of the provided materials. The overview should not merely summarize each section but should reconstruct the content into a logically structured and compelling exposition of the central themes, arguments, and discussions.

      Your overview should:

      - Present the topic, thesis, and core arguments in a clear and logically progressive manner.
      - Synthesize key ideas, discussions, and counterarguments into a cohesive narrative.
      - Reorganize content if necessary to enhance clarity, argumentative strength, and thematic development.
      - Ensure fluid transitions between concepts and sections to guide the reader effectively.
      - Capture nuance, theoretical depth, and the broader implications of the discussion."""
    )

    prompt = f"""
    Generate an essay overview based on the following:

    Thesis:
    {thesis}

    Arguments:
    {arguments}

    Context:
    {context}

    Discussion:
    {discussion}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "overview": "string",
      "sections": [
        {{
          "title": "string",
          "summary": "string"
        }}
      ]
    }}

    - The "overview" field should contain a 2-3 sentence high-level summary of the essay.
    - The "sections" array should include each major section of the essay, with a brief summary of its purpose and content.
    - Ensure clarity, coherence, and proper structuring.
    """

    return call_llm(prompt, system_prompt)

def structure_agent(overview):
    system_prompt = (
        "You are an academic writing strategist in the field of philosophy leading a team of professional philosophers "
        "in the authorship of an academic paper. Your task is to create a structured outline for a philosophy essay "
        "based on the provided overview. The outline must be logically structured, ensuring coherence and flow. However, "
        "the structure need not follow the input structure. Rather, the structure should optimally present the thesis in a "
        "logical and cohesive progression of detailed academic exposition and discussion to present and discuss the topic"
        "purpose of drawing meaningful conclusions regarding the thesis. "
        "It should be formatted as a JSON object that can be parsed programmatically."
    )

    prompt = f"""
    Generate a structured outline for the following essay overview:

    {overview}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "title": "string",
      "sections": [
        {{
          "title": "string",
          "summary": "string",
          "paragraphs": [
            {{
              "id": "integer",
              "topic": "string",
              "details": "string"
            }}
          ]
        }}
      ]
    }}

    - The "title" field should contain the title of the essay.
    - Each "sections" object should include a section title and a brief summary of its purpose.
    - Each section should contain a "paragraphs" array with numbered paragraph entries, including a topic and a description of what it should cover.
    - Ensure that the outline provides a logical flow from introduction to conclusion.
    """

    return call_llm(prompt, system_prompt)

import re

def split_paragraph(paragraph_text):
    """Splits a paragraph into sentences while avoiding splitting after common abbreviations."""
    if not paragraph_text.strip():
        return []  # Return empty list for empty input

    # Define common abbreviations that should not cause sentence splits
    abbreviations = {
        "Dr.": "Dr<abbr>",
        "Mr.": "Mr<abbr>",
        "Ms.": "Ms<abbr>",
        "Mrs.": "Mrs<abbr>",
        "Jr.": "Jr<abbr>",
        "Sr.": "Sr<abbr>",
        "vs.": "vs<abbr>",
        "etc.": "etc<abbr>",
        "e.g.": "eg<abbr>",
        "i.e.": "ie<abbr>"
    }

    # Step 1: Temporarily replace abbreviations
    for abbr, placeholder in abbreviations.items():
        paragraph_text = paragraph_text.replace(abbr, placeholder)

    # Step 2: Split sentences using a simple regex
    sentences = re.split(r'(?<=[.!?])\s+', paragraph_text.strip())

    # Step 3: Restore abbreviations
    sentences = [sentence.replace("<abbr>", ".") for sentence in sentences]

    # Return structured sentence objects
    return [{"id": i, "text": sentence.strip()} for i, sentence in enumerate(sentences) if sentence.strip()]

def drafting_agent(section_outline, overview):
    system_prompt = (
        "You are an academic writer. Your task is to write a well-structured and logically coherent paragraph "
        "based on the provided section outline, while ensuring it aligns with the overall structure and flow "
        "outlined in the essay overview. The paragraph should follow academic standards for clarity and precision. "
        "Ensure that the paragraph remains consistent with the topic and details provided in the outline."
    )

    prompt = f"""
    Generate a paragraph based on the following outline and essay overview:

    Overview:
    {overview}

    Section Outline:
    {section_outline}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "paragraph": "string"
    }}

    - The paragraph should be clear, well-structured, and aligned with the topic and details provided.
    - Ensure logical flow and coherence within the paragraph.
    - Maintain consistency with the essay's overall structure as outlined in the overview.
    """

    return call_llm(prompt, system_prompt)

def gross_revision_agent(paragraph, thesis, argument, section_outline):
    system_prompt = (
        """You are a philosophy editor specializing in deepening academic writing. "
        "Your task is to revise the provided paragraph to enhance its philosophical depth, detail, exposition, and clarity. "
        "Strengthen the argument by expanding key points, improving explanations, and incorporating additional support where necessary. "
        "Ensure that every claim is well-articulated, logically developed, and contextualized within the broader discussion. "
        "Focus on increasing precision and depth without altering the intended meaning or introducing unrelated ideas. "
        "Clarify abstract or ambiguous statements, reinforce logical connections, and provide additional exposition where needed to make the argument more rigorous and comprehensive. "
        "Maintain an academic tone, ensuring that the paragraph aligns seamlessly with the section’s argument and the overarching thesis of the essay."""
    )

    prompt = f"""
    Revise the following paragraph for deeper philosophical engagement, refining arguments, and adding necessary detail.

    Thesis:
    {thesis}

    Section Outline:
    {section_outline}

    Supporting Argument:
    {argument}

    Original Paragraph:
    {paragraph}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "revised_paragraph": "string"
    }}

    - Ensure the revised paragraph maintains logical coherence with the thesis and section argument.
    - Improve depth, precision, and clarity in philosophical reasoning.
    - Preserve the intended meaning while enhancing readability and engagement.
    """

    return call_llm(prompt, system_prompt)

def section_instruction_agent(section, thesis, argument):
    system_prompt = (
        "You are an academic writing editor. Your task is to analyze a section of a philosophy essay "
        "and generate paragraph-specific revision instructions. Identify redundancies, improve logical flow, "
        "and suggest wording adjustments. You may not restructure the section or recommend the addition or "
        "removal of full paragraphs. Do NOT rewrite the section—only provide structured revision guidance."
    )

    prompt = f"""
    Analyze the following section and generate structured revision instructions for each paragraph.

    Thesis:
    {thesis}

    Argument:
    {argument}

    Section Title: {section["title"]}

    Section Content:
    {" ".join(paragraph["text"] for paragraph in section["paragraphs"])}

    Your response must be formatted as a JSON object:
    {{
      "revision_instructions": [
        {{
          "id": "integer",
          "instructions": "string"
        }}
      ]
    }}

    - Identify redundant ideas across paragraphs.
    - Improve logical flow between paragraphs.
    - Suggest rewording for clarity and conciseness.
    - Do NOT rewrite the section, only provide structured revision guidance.
    """

    return call_llm(prompt, system_prompt)

def paragraph_revision_agent(paragraph, instructions, thesis, argument):
    system_prompt = (
        "You are an academic writing editor. Your task is to refine a single paragraph based on structured revision instructions. "
        "Ensure clarity, conciseness, and logical alignment. Apply the suggested revisions, but do NOT remove the paragraph."
    )

    prompt = f"""
    Refine the following paragraph based on the provided revision instructions.

    Thesis:
    {thesis}

    Argument:
    {argument}

    Revision Instructions:
    {instructions}

    Original Paragraph:
    {paragraph["text"]}

    Your response must be formatted as a JSON object:
    {{
      "refined_paragraph": "string"
    }}

    - Implement the suggested improvements while maintaining the paragraph's meaning.
    - Improve clarity, conciseness, and logical flow.
    - Do NOT remove the paragraph.
    """

    return call_llm(prompt, system_prompt)


def fine_revision_agent(sentence, paragraph_context, section_outline):
    system_prompt = (
        "You are a professional proofreader specializing in academic philosophy. Your task is to refine the given sentence "
        "to improve clarity, grammar, and conciseness while ensuring it maintains its intended meaning. "
        "Additionally, ensure the sentence flows well within the paragraph and aligns with the section’s structure."
    )

    prompt = f"""
    Revise the following sentence for clarity, grammatical accuracy, and conciseness while preserving its original meaning.

    Section Outline:
    {section_outline}

    Paragraph Context:
    {paragraph_context}

    Original Sentence:
    {sentence}

    Your response must be formatted as a JSON object with the following structure:

    {{
      "revised_sentence": "string"
    }}

    - Do not include any additional text, explanations, or formatting such as Markdown.
    - Ensure grammatical correctness and conciseness.
    - Maintain the intended meaning of the sentence.
    - Improve readability while ensuring a natural flow within the paragraph.
    """

    return call_llm(prompt, system_prompt)

def section_refinement_agent(section, thesis, argument):
    system_prompt = (
        "You are an academic writing editor specializing in philosophical essays. Your task is to refine the following section "
        "by removing redundant content, ensuring coherence between paragraphs, and maintaining logical flow. "
        "Eliminate unnecessary repetition while preserving depth and rigor in argumentation."
    )

    prompt = f"""
    Refine the following section of a philosophy essay to improve coherence and eliminate redundancy.

    Thesis:
    {thesis}

    Argument:
    {argument}

    Section Title: {section["title"]}

    Section Content:
    {" ".join(paragraph["text"] for paragraph in section["paragraphs"])}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "refined_section": [
        {{
          "id": "integer",
          "text": "string"
        }}
      ]
    }}

    - Maintain logical flow and academic rigor.
    - Ensure each paragraph contributes uniquely to the argument.
    - Improve readability by reducing unnecessary repetition.
    - Keep the response strictly formatted as JSON.
    """

    return call_llm(prompt, system_prompt)

def citation_scanning_agent(paragraph, section_outline, thesis, context):
    system_prompt = (
        "You are a citation analyst specializing in academic philosophy. You are given details of an academic philosophy paper that lacks citations. "
        "Your task is to identify all sentences in the given paragraph that will require attribution to previous work."
        "Your report should include the sentence, and to whom the concept, idea, or work should be attributed, and why."
        "If a sentence is considered general knowledge or is original research unique to the paper you're reading, then no attribution is necessary, and it can be left out of your report."
        "You must follow report formatting guidelines."
    )

    prompt = f"""
    Identify all sentences in the following paragraph that require citations and specify the type of source needed.

    Thesis:
    {thesis}

    Section Outline:
    {section_outline}

    Context of the paper (these sources have been noted from the paper's thesis and planning, and may differ from those in he paragraph provided for analysis):
    {context}

    Paragraph to analyze:
    {paragraph}

    Your response must be formatted as a JSON object with the following structure:
    {{
      "citations": [
        {{
          "sentence": "string",
          "identity": "string",
          "reason": "string",
        }}
      ]
    }}

    - The "sentence" field should contain the exact sentence that requires attribution.
    - The "identity" field should the name of the author or entity to whom attribution is required.
    - The "reason" field should explain why a citation is needed and how the sentence relates to the previous work of the author or entity in the identity field.
    - Only include sentences that explicitly require citations based on academic standards. General knowledge and original research do not need to be attributed.
    - The suggested context includes known philosophical works and thinkers relevant to the thesis, but citations are not limited to these sources.
    """

    return call_llm(prompt, system_prompt)

def citation_insertion_agent(paragraph, citations):
    system_prompt = (
        "You are a reference manager specializing in academic philosophy. Your task is to insert parenthetical citations "
        "of the form (LastName) into the given paragraph, ensuring proper attribution in the result. Utilize the provided citation suggestions "
        "as guidelines. If you notice an error in the provided citations, you should use the correct attribution."
        "For each sentence, if the source is cited in the sentence by name, you do not need to add a parenthetical citation at the end of the sentence."
    )

    prompt = f"""
    Insert appropriate citations into the following paragraph using the provided citation suggestions.

    Paragraph:
    {paragraph}

    Citation Suggestions:
    {citations}

    Your response must be formatted as a JSON object with the following structure:

    {{
      "cited_paragraph": "string"
    }}

    - Ensure citations are placed in appropriate locations.
    - Use the format (LastName) for the appropriate attributable entity.
    - Do not include any additional information like works or year in the parenthetical citation. ONLY the last name of the identity receiving the attribution should appear in the parenthetical citation.
    """
    return call_llm(prompt, system_prompt)

def citation_extraction_agent(paragraph):
    system_prompt = (
        "You are a reference extraction assistant. Your task is to extract all citations from the provided paragraph."
        "and return them as citations in Chicago format."
    )

    prompt = f"""
    Extract citations from the following paragraph:

    {paragraph["text"]}

    Your response must be formatted as a JSON object with the following structure:

    {{
      "citations": [
        {{
          "identity": "string",
          "note": "string",
        }}
      ]
    }}

    - The "identity" field should the name of the author or entity to whom attribution is made.
    - The "note" fielr should be a brief sentence noting why the attribution was made to the author or entity.
    - Only extract unique citations (do not repeat within the paragraph).
    - Do not add any extra text outside of the JSON object.
    """

    return call_llm(prompt, system_prompt)

def works_cited_aggregation_agent(citation_list,thesis):
    system_prompt = (
        "You are an academic reference organizer. Your task is to consolidate a list of extracted citations into a structured, unique works cited list. "
        "Each cited work must be unique, with duplicates removed. Ensure proper academic formatting (e.g., MLA, APA, or Chicago style)."
    )

    prompt = f"""

    Here is the Thesis of the paper:

    Thesis:
    {thesis}

    Consolidate the following extracted citations into a unique works cited list.

    Citation List:
    {citation_list}

    Your response must be formatted as a JSON object:
    {{
      "works_cited": [
        {{
          "identity": "string",
          "description": "string",
          "relevance":"relevance"
        }}
      ]
    }}

    - The "identity" field should the name of the author or entity to whom attribution is made.
    - The "description" field gives a brief overview of the author or entity and the totality of their work.
    - The "relevance" field gives a few sentences on how the author or entity
    - Standardize the citation formatting for consistency.
    - Do not include any extra text outside the JSON object.
    """

    return call_llm(prompt, system_prompt)


def abstract_agent(essay_json):
    system_prompt = (
        "You are an academic summarizer specializing in philosophy. Your task is to write a concise, well-structured abstract "
        "that effectively summarizes the entire essay, including the thesis, key arguments, philosophical context, discussion points, and conclusion. "
        "Ensure the abstract is engaging, clear, and informative while adhering to academic standards."
    )

    # Step 1: Summarize each section separately to reduce input size
    section_summaries = []
    for section in essay_json["sections"]:
        section_prompt = f"""
        Summarize the following section of a philosophy essay:

        Section Title: {section['title']}

        Section Content:
        {section['summary']}

        Paragraphs:
        {" ".join(paragraph['text'] for paragraph in section['paragraphs'])}

        Your response must be formatted as a JSON object:
        {{
          "section_summary": "string"
        }}
        """
        summary_response = call_llm(section_prompt, system_prompt)
        section_summary = parse_json_with_validation(summary_response).get("section_summary", "")
        section_summaries.append({"title": section["title"], "summary": section_summary})

    # Step 2: Use section summaries for the final abstract generation
    abstract_prompt = f"""
    Below are the key components of a philosophy essay. Your task is to generate a concise abstract.

    Title: {essay_json["title"]}

    Thesis:
    {essay_json["thesis"]}

    Context:
    {essay_json["context"]}

    Arguments:
    {essay_json["arguments"]}

    Discussion:
    {essay_json["discussion"]}

    Section Summaries:
    """
    for section in section_summaries:
        abstract_prompt += f"Section: {section['title']}\nSummary: {section['summary']}\n\n"

    abstract_prompt += """
    Your response must be formatted as a JSON object with the following structure:
    {
      "abstract": "string"
    }

    - Ensure the abstract is concise (150-250 words), engaging, and informative.
    - Clearly summarize the thesis, major arguments, counterarguments, and philosophical significance.
    - Maintain logical coherence and clarity for an academic audience.
    """

    return call_llm(abstract_prompt, system_prompt)



def json_fixing_agent(response_text):
    """
    Uses an LLM to clean and fix malformed JSON structures, ensuring proper formatting and structure.
    This function is designed to be called iteratively within a validation loop.
    """

    system_prompt = (
        "You are an expert JSON repair agent. Your task is to extract, clean, and correct any malformed JSON "
        "found in a given text response. The JSON may have missing brackets, incorrect formatting, or extraneous text."
        "Your response must contain ONLY valid JSON with no extra text, explanations, or comments."
    )

    prompt = f"""
    The following text contains malformed JSON that may have structural errors or extra text.
    Extract and correct the JSON, ensuring it is properly formatted and syntactically valid.

    Response to fix:
    {response_text}

    Return ONLY the corrected JSON.
    """

    fixed_response = call_llm(prompt, system_prompt)

    return fixed_response

def parse_json_with_validation(response_text):
    """
    Attempts to parse a JSON response by running it through the JSON Fixing Agent iteratively
    until it parses successfully or reaches a retry limit.
    """
    global MAX_JSON_ATTEMPTS
    attempts = 0

    while attempts < MAX_JSON_ATTEMPTS:
        try:
            # Remove markdown-style code blocks if they exist
            response_text = re.sub(r'```json|```', '', response_text).strip()

            # Extract JSON block safely (no lookbehind)
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if not json_match:
                raise json.JSONDecodeError("No valid JSON found", response_text, 0)

            cleaned_json_text = json_match.group(0)
            parsed_json = json.loads(cleaned_json_text)

            #print(f"Successfully parsed JSON on attempt {attempts + 1}")
            return parsed_json
        except json.JSONDecodeError:
            print(f"JSON parsing failed. Running JSON Fixing Agent...")
            response_text = json_fixing_agent(response_text)
            attempts += 1

    print("Failed to parse JSON after maximum attempts.")
    return {"error": "Failed to parse JSON after multiple attempts."}

def convert_json_to_plaintext(essay_json):
    """
    Converts the final structured JSON essay into a plain-text readable format.
    """
    plaintext = ""
    plaintext += f"{essay_json['title']}\n\n"
    plaintext += f"Abstract\n{essay_json['abstract']}\n\n"

    plaintext += "Body\n\n"
    for section in essay_json['sections']:
        plaintext += f"{section['title']}\n\n"
        for paragraph in section['paragraphs']:
            plaintext += f"{paragraph['text']}\n\n"

    plaintext += "Works Cited\n\n"
    for philosopher in essay_json['context']['philosophers']:
        plaintext += f"{philosopher['name']}. *{philosopher['work']}*. {philosopher['relevance']}.\n"
    plaintext += "\n"
    for concept in essay_json['context']['concepts']:
        plaintext += f"{concept['name']}. {concept['definition']}. {concept['relevance']}.\n"
    plaintext += "\n"

    return plaintext

MAX_JSON_ATTEMPTS = 5

def generate_philosophy_essay(topic, fine_revisions=False):
    """
    Generates a full philosophy essay based on the given topic.
    The user can choose to include or skip fine sentence-by-sentence revisions.
    """

    print("Generating thesis...")
    thesis_response = thesis_agent(topic)
    thesis = parse_json_with_validation(thesis_response)["thesis"]

    print("Gathering context...")
    context_response = context_agent(thesis)
    context = parse_json_with_validation(context_response)

    print("Constructing arguments...")
    argument_response = argument_agent(thesis)
    arguments = parse_json_with_validation(argument_response)

    print("Facilitating discussion...")
    discussion_response = discussion_agent(arguments,context)
    discussion = parse_json_with_validation(discussion_response)

    print("Creating overview...")
    overview_response = overview_agent(thesis, arguments, context, discussion)
    overview = parse_json_with_validation(overview_response)

    print("Structuring the essay...")
    structure_response = structure_agent(overview)
    structure = parse_json_with_validation(structure_response)

    print("Drafting paragraphs...")
    for section in structure["sections"]:
        for paragraph in section["paragraphs"]:
            draft_response = drafting_agent(paragraph, overview)
            paragraph["text"] = parse_json_with_validation(draft_response)["paragraph"]

    print("Performing gross revision...")
    for section in structure["sections"]:
        for paragraph in section["paragraphs"]:
            revised_response = gross_revision_agent(paragraph["text"], thesis, arguments, section)
            paragraph["text"] = parse_json_with_validation(revised_response)["revised_paragraph"]

    print("Refining sections for coherence and eliminating redundancy...")
    for section in structure["sections"]:
        # Step 1: Generate revision instructions for the entire section
        instruction_response = section_instruction_agent(section, thesis, arguments)
        revision_instructions = parse_json_with_validation(instruction_response)["revision_instructions"]

        # Step 2: Apply paragraph-by-paragraph revisions using instructions
        for i, paragraph in enumerate(section["paragraphs"]):
            instruction_text = revision_instructions[i]["instructions"] if i < len(revision_instructions) else "No specific revision needed."

            refined_response = paragraph_revision_agent(paragraph, instruction_text, thesis, arguments)
            section["paragraphs"][i]["text"] = parse_json_with_validation(refined_response)["refined_paragraph"]

        if fine_revisions:
            print("Performing fine revision (this may take a while)...")
            for section in structure["sections"]:
                for paragraph in section["paragraphs"]:
                    sentences = split_paragraph(paragraph["text"])
                    for sentence in sentences:
                        fine_response = fine_revision_agent(sentence["text"], paragraph["text"], section)
                        sentence["text"] = parse_json_with_validation(fine_response)["revised_sentence"]
                    paragraph["text"] = merge_sentences(sentences)

    print("Scanning for citations...")
    works_cited = {}  # Track citations for the final Works Cited section

    for section in structure["sections"]:
        for paragraph in section["paragraphs"]:
            citation_scan_response = citation_scanning_agent(paragraph["text"], section, thesis, context)
            citation_suggestions = parse_json_with_validation(citation_scan_response)

            citation_insert_response = citation_insertion_agent(paragraph["text"], citation_suggestions)

            # Validate the response
            citation_insert_response = parse_json_with_validation(citation_insert_response)

            cited_paragraph = citation_insert_response["cited_paragraph"]
            paragraph["text"] = cited_paragraph

            # Track citations in the Works Cited dictionary
            for reference in citation_insert_response.get("references", []):
                key = (reference["author"], reference["work"], reference["year"])
                if key not in works_cited:
                    works_cited[key] = reference["citation_format"]

    print("Generating abstract...")
    essay_json = {
        "title": topic,
        "thesis": thesis,
        "context": context,
        "arguments": arguments,
        "discussion": discussion,
        "sections": structure["sections"]
    }

    abstract_response = abstract_agent(essay_json)
    abstract = parse_json_with_validation(abstract_response)["abstract"]

    print("Extracting attributable ideas from paragraphs...")
    citation_entries = []

    for section in structure["sections"]:
        for paragraph in section["paragraphs"]:
            extraction_response = citation_extraction_agent(paragraph)
            extracted_citations = parse_json_with_validation(extraction_response)["citations"]
            citation_entries.extend(extracted_citations)

    print("Aggregating attributions...")
    aggregation_response = works_cited_aggregation_agent(citation_entries,thesis)
    unique_citations = parse_json_with_validation(aggregation_response)["works_cited"]

    # Compile the final Attributions section
    works_cited_list = "\n".join(
        f"{entry['identity']}. *{entry['description']}*. {entry['relevance']}." for entry in unique_citations
    )
    essay_json["works_cited"] = works_cited_list
    print("Attribution compiled.")

    # Print Title
    print(essay_json['title'])
    print("")  # Blank line for spacing

    # Print Abstract
    print("Abstract")
    print("")
    print(abstract)
    print("")  # Blank line for spacing

    # Print Main Essay Body
    print("Essay Body")
    print("")

    for section in essay_json["sections"]:
        # Print Section Title
        print(section['title'])
        print("")  # Blank line for spacing

        # Print Each Paragraph in the Section
        for paragraph in section["paragraphs"]:
            print(paragraph['text'])
            print("")  # Blank line between paragraphs

    # Print Works Cited
    print("Attribution")
    print("")
    print(works_cited_list)
    print("")  # Final blank line for clean output
    #return final_essay

client = Together(api_key=KEY)
generate_philosophy_essay(Topic, fine_revisions=False)